In [2]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python flask  # type: ignore

In [2]:
from kafka import KafkaProducer

topic = "smoke-test"
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
# Note: The data you send must be binary
producer.send(topic, b"Hello World!").get(timeout=30)
# Note: The data you send must be binary
producer.send(topic, b"Foo").get(timeout=30)
producer.send(topic, b"Bar").get(timeout=30)

RecordMetadata(topic='smoke-test', partition=0, topic_partition=TopicPartition(topic='smoke-test', partition=0), offset=2, timestamp=1637862851980, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=3, serialized_header_size=-1)

In [3]:
from IPython.display import display, clear_output
from time import sleep

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'
os.environ['SPARK_OPTS'] = "--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=warn"


import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[2]').getOrCreate()
sc = spark.sparkContext

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers","localhost:9092")
    .option("subscribe", "smoke-test")
    .option("startingOffsets", "earliest")
    .load()
)

smoke_test_query = (
    df.writeStream                  # Create an output stream for the query
    .outputMode("append")           # Add each msg to end of "streaming table"
    .format("memory")               # Write output stream to Spark table
    .queryName("test_query") 
    .start()
)

# Wait until query runs for a bit
sleep(10)

# Display status of query
display(smoke_test_query.status)

# Show result in Jupyter Notebook
display(
    spark.table("test_query").toPandas()
)

# Stop query
smoke_test_query.stop()


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-38f23ba4-2a1d-4ed8-ad61-18acc10e556b;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.2.0/spark-sql-kafka-0-10_2.12-3.2.0.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0!spark-sql-kafka-0-10_2.12.jar (76ms)
downloading

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[72, 101, 108, 108, 111, 32, 87, 111, 114, 108...",smoke-test,0,0,2021-11-25 17:54:11.948,0
1,None,"[70, 111, 111]",smoke-test,0,1,2021-11-25 17:54:11.977,0
2,None,"[66, 97, 114]",smoke-test,0,2,2021-11-25 17:54:11.980,0
